# Combining Affiliations
Affiliations were collected in two parts: 1940 to 2010 and 2011 to 2020. Although the two sets are individually consolidated, they are not fully consolidated against each other. Before this can happen:

- The 1940 to 2010 file (pre_2010) needs to be restructured
- Both files also contain data fields that are not necessary

The variation due to the different individuals who consolidated the two data files. The following code simply combines the two data sets, checks for logical errors (eg: row contains data for 6 authors when there are only 5) and simplies the available fields to the bare minimum.

Input files:
- affiliations file pre-2010
- affiliations file post-2010

Output files:
- pre-2010 file with all fields and corrections
- post-2010 file with all fields and corrections
- combined file with all fields and filled corrections
- combined file of subset of fields

In [1179]:
import pandas as pd
import time
import json
import re
pd.set_option('display.max_colwidth', 200)

In [1180]:
def save_json(data, file_path, encoding_type):
    try:
        with open(file_path, "w", encoding=encoding_type) as json_file:
            json.dump(data, json_file, ensure_ascii=False, indent=4)
        print(f"JSON data successfully saved to {file_path} with {encoding_type} encoding.")
    except IOError as e:
        print(f"Error saving JSON data: {e}")

In [1181]:
base_path="/Users/sijiawu/Work/Thesis/Data/"
content_ex=['MISC',  'Discussion', 'Review', 'Review2']
content=['Article', 'Comment', 'Reply', 'Rejoinder',"Errata"]
jid=["aer","ecta","jpe","qje","res"]

In [1182]:
pre_2010=pd.read_excel(base_path+"Affiliations/Affiliations_cleaned_1940_2010.xlsx")
pre_2010_remnant=pd.read_excel(base_path+"Affiliations/affs_remnant_mixed_final.xlsx")
post_2010_init=pd.read_excel(base_path+"Affiliations/Affiliations_cleaned_2011_2020_alt.xlsx")
post_2010_remnant=pd.read_excel(base_path+"Affiliations/affs_remnants_alt.xlsx")

j_data=pd.read_pickle(base_path+"Combined/020_merged_proc_scopus_inception_with_auth_split_2020.pkl")

In [1183]:
pre_2010_remnant.shape

(5127, 18)

In [1184]:
pre_2010.shape

(22789, 40)

In [1185]:
post_2010_init.shape

(11507, 20)

In [1186]:
post_2010_remnant.shape

(3038, 25)

In [1187]:
rnames=pre_2010.columns
rnames

Index(['Journal_Name', 'Authors_count', 'Authors_list', 'authors',
       'Article_ID', 'content_type', 'stable_url', 'year', 'Ref_type',
       'ref_string', 'ref_string_clean1', 'Screened', 'Manual',
       '0_Auth_affs_0', '0_Auth_affs_1', '0_Auth_affs_2', '0_Auth_affs_3',
       '0_Auth_affs_4', '1_Auth_affs_0', '1_Auth_affs_1', '1_Auth_affs_2',
       '1_Auth_affs_3', '1_Auth_affs_4', '2_Auth_affs_0', '2_Auth_affs_1',
       '2_Auth_affs_2', '2_Auth_affs_3', '2_Auth_affs_4', '3_Auth_affs_0',
       '3_Auth_affs_1', '3_Auth_affs_2', '3_Auth_affs_3', '3_Auth_affs_4',
       '4_Auth_affs_0', '4_Auth_affs_1', '4_Auth_affs_2', '4_Auth_affs_3',
       '5_Auth_affs_0', '6_Auth_affs_0', '7_Auth_affs_0'],
      dtype='object')

In [1188]:
dict= {}
issues=[]
missing_auth=[]
a=1

for i in pre_2010.index:
    stable_cols=["Journal_Name", 
                 "Authors_count", 
                 "Authors_list", 
                 "authors", 
                 "Article_ID", 
                 "content_type", 
                 "stable_url", "year", "Ref_type", "ref_string", "ref_string_clean1", 'Screened', 'Manual']
    auth_list= pre_2010.loc[i, ["Authors_list"]]
    authors= pre_2010.loc[i, ["authors"]]
    count= pre_2010.loc[i, "Authors_count"]
    temp=pre_2010.loc[i,stable_cols].to_dict()

    m=None    
    if pd.isna(auth_list.values[0]):
        if (pd.isna(authors.values)==False):
            # print(i)
            # print(authors)
            x=authors.values[0].split(",")
            m=[]
            for k in x:
                y=k.split(" And ")
                m=m+y
            missing_auth.append(i)
            # print(auth_list.values[0])
        else:
            issues.append(i)
            continue

    proc_auth=-1
    if m==None:
        proc_auth=auth_list.values[0][2:-2].split(",")
    else:
        proc_auth=m

    if len(proc_auth)!=int(pre_2010.loc[i, "Authors_count"]):
        print(i)
        print(authors)

    temp_dict={}
    for k in range(len(proc_auth)):
        temp_dict[k]=proc_auth[k]
    # print(i)

    for j in rnames:

        if (pd.isna(pre_2010.loc[i,j])==False) and (j not in stable_cols):
            entry={}
            # print('"'+j+'": '+pre_2010.loc[i,j])
            label=j.split("_")
            entry["auth_num"]=int(label[0])
            entry["aff_num"]=(int(label[-1])+1)
            entry=entry|temp
            entry["aff_main_final"]=pre_2010.loc[i,j]
            entry["author"]=temp_dict[int(label[0])]
            # print(entry)

            dict[a]=entry
            a=a+1
            

In [1189]:
pre_2010_init=pd.DataFrame(dict).transpose()
pre_2010_init.rename(columns={'Journal_Name': "journal", 
                            'Article_ID': "id", 
                            'Authors_count':'auth_count', 
                            'Authors_list':'auth_list',  
                            'Ref_type': "ref_type", 
                            'ref_string_clean1':'ref_string_cleaner' , 
                            'Screened': 'screened', 
                            'Manual':'manual'}, inplace=True)

In [1190]:
pre_2010_init[["ref_string", "ref_string_cleaner"]].head(1)

,ref_string,ref_string_cleaner
1,MR. KEYNES AND MR. MARX 135\n\nabove-noted differences between Keynes and Marx with respect to the depend-\nence of the marginal efficiency of capital on the rate of real wages and on the\npropens...,Mr. Keynes And Mr. Marx 135\n\nAbove-Noted Differences Between Keynes And Marx With Respect To The Depend-\nEnce Of The Marginal Efficiency Of Capital On The Rate Of Real Wages And On The\nPropens...


In [1191]:
pre_2010_remnant.head()

,pdf_url,hitId,worker,id,year,title,journal,pages,vol,number,author,affiliation,auth_num,aff_num,aff_clean,aff_main,add_sub,aff_main_final
0,https://myawsbucket-1231.s3.eu-west-3.amazonaws.com/missing/1832157_wo_cover.pdf,3M47JKRKD4WS8FZ62ZABUD18TU286O,A345TDMHP3DQ3G,1832157,1967,Major Issues in the Regulation of Financial Institutions: Comment,Journal of Political Economy,509-511,75,4,Charles E. Walker,American Bankers Association,0,1,NaN,NaN,NaN,American Bankers Association
1,https://myawsbucket-1231.s3.eu-west-3.amazonaws.com/missing/1803922_wo_cover.pdf,3A520CCNXUVA4IL05FDN2SZG0GRAEG,A3HMGR8WPY9OPS,1803922,1983,Commodities and Capital: Prices and Quantities,The American Economic Review,1-16,73,1,Gardner Ackley,American Economic Association,0,1,NaN,NaN,NaN,American Economic Association
2,https://myawsbucket-1231.s3.eu-west-3.amazonaws.com/missing/1808715_wo_cover.pdf,3QREJ3J44ASUM14EYSDG91SWEPLLKS,A345TDMHP3DQ3G,1808715,1979,Monopoly Output under Alternative Spatial Pricing Techniques: Reply,The American Economic Review,680-681,69,4,H. Ohta,Aoyama Gakuin University,1,1,NaN,NaN,NaN,Aoyama Gakuin University
3,https://myawsbucket-1231.s3.eu-west-3.amazonaws.com/missing/1809704_wo_cover.pdf,3ODOP6T3BZF37DT33U8VNO4RB3542X,A3HMGR8WPY9OPS,1809704,1978,Related Market Conditions and Interindustrial Mergers: Reply,The American Economic Review,228-230,68,1,H. Ohta,Aoyama Gakuin University,1,1,NaN,NaN,NaN,Aoyama Gakuin University
4,https://myawsbucket-1231.s3.eu-west-3.amazonaws.com/missing/1813335_wo_cover.pdf,3LG268AV4F123Z7H7QZ9XIOQ0KFERS,A3NH3VJF04DBJX,1813335,1974,Price Discrimination by Regulated Motor Carriers: Comment,The American Economic Review,780-784,64,4,H. Ohta,Aoyama Gakuin University,2,1,NaN,NaN,NaN,Aoyama Gakuin University


In [1192]:
pre_2010_remnant["manual"]="MTURK"
pre_2010_remnant['stable_url']="https://www.jstor.org/stable/"+pre_2010_remnant.pdf_url.str.split('/').str[-1].str.split('_').str[0]
pre_2010_remnant['screened']='y'

pre_2010_df=pd.concat([pre_2010_init, pre_2010_remnant], axis=0).reset_index(drop=True)

In [1193]:
pre_2010_df.to_excel(base_path+"Affiliations/aff_pre_2010.xlsx", index=False)
pre_2010_df.to_pickle(base_path+"Affiliations/aff_pre_2010.pkl")

In [1194]:
pre_2010_df[pre_2010_df["aff_main_final"].isna()].shape

(21, 28)

In [1195]:
post_2010_init.head()

,id,year,title,journal,pages,vol,number,author,affiliation,auth_num,aff_num,aff_cleaner,aff_cleaner_final,aff_main_final,aff_subunit_final,aff_department_final,alt_final,country,is_business_school,is_institute
0,41724643,2012,Political Aid Cycles,american economic review,3516-3530,102,7,Michael Faye,"128 East 62nd Street, New York, NY",0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,42920725,2014,When the Levee Breaks: Black Migration and Economic Development in the American South,american economic review,963-990,104,3,Suresh Naidu,1405 IAB MC 3328,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,26863278,2020,Way Down in the Hole: Adaptation to Long-Term Water Loss in Rural India,american economic review,200-224,110,1,David Blakeslee,Abu Dhabi,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,41407067,2011,Learning and Information Aggregation in an Exit Game,review of economic studies,1426-1461,78,4,JUUSO VÄLIMÄKI,Aalto University,1,1,Aalto University,Aalto University,Aalto University,0,0,Aalto University,NaN,NaN,NaN
4,26549901,2016,Politicians’ Luck of the Draw: Evidence from the Spanish Christmas Lottery,journal of political economy,1269-1294,124,5,Manuel Bagues,Aalto University,0,1,Aalto University,Aalto University,Aalto University,0,0,Aalto University,NaN,NaN,NaN


In [1196]:
post_2010_remnant=post_2010_remnant[['id', 'year', 'title', 'journal', 'pages', 'vol', 'number', 'author',
       'affiliation', 'auth_num', 'aff_num', 'aff_cleaner',
       'aff_cleaner_final', 'aff_main_final', 'aff_subunit_final',
       'aff_department_final', 'alt_final', 'country', 'is_business_school',
       'is_institute']]

post_2010=pd.concat([post_2010_remnant,post_2010_init]).reset_index(drop=True)

post_2010['manual']="MTURK"
post_2010['screened']="y"


In [1197]:
check_post=[
    {"id":43616932,"auth_num":1,"aff_num":1,"affiliation":"Dept. of Economics and Related Studies, University of York", "aff_cleaner":"University of York","aff_cleaner_final":"University of York","aff_main_final":"University of York"},
    {"id":'qjz003',"auth_num":0,"aff_num":1,"affiliation":"MASSACHUSETTS INSTITUTE OF TECHNOLOGY", "aff_cleaner":"mit","aff_cleaner_final":"mit","aff_main_final":"mit"},
    {"id":'qjz003',"auth_num":0,"aff_num":3,"affiliation":"Centre for Economic and Policy Research", "aff_cleaner":"Centre for Economic and Policy Research","aff_cleaner_final":"Centre for Economic and Policy Research","aff_main_final":"Centre for Economic and Policy Research"},
    {"id":'qjz003',"auth_num":1,"aff_num":1,"affiliation":"MASSACHUSETTS INSTITUTE OF TECHNOLOGY", "aff_cleaner":"mit","aff_cleaner_final":"mit","aff_main_final":"mit"},
    {"id":'qjz003',"auth_num":1,"aff_num":3,"affiliation":"Centre for Economic and Policy Research", "aff_cleaner":"Centre for Economic and Policy Research","aff_cleaner_final":"Centre for Economic and Policy Research","aff_main_final":"Centre for Economic and Policy Research"},
    {"id":26863278,"auth_num":0,"aff_num":1,"affiliation":"new york university - abu dhabi", "aff_cleaner":"new york university - abu dhabi","aff_cleaner_final":"new york university - abu dhabi","aff_main_final":"new york university - abu dhabi"}
]
comps=[]
for i in check_post:
    ind=list(post_2010[(post_2010["id"]==i["id"])& (post_2010["auth_num"]==i["auth_num"])& (post_2010["aff_num"]==i["aff_num"])].index)
    if len(ind)>2:
        print("issue")
        continue
    if len(ind)==0:
        ind=list(post_2010[(post_2010["id"]==i["id"])& (post_2010["auth_num"]==i["auth_num"])].index)[0]
        tmp=post_2010.loc[ind].to_dict()
        for j in i.keys():
            tmp[j]=i[j]
        comps.append(tmp)
    else:
        for j in i.keys():
            post_2010.loc[ind,j]=i[j]

post_2010=pd.concat([pd.DataFrame(comps),post_2010]).reset_index(drop=True)

In [1198]:
post_2010=post_2010[post_2010["aff_main_final"].isna()==False].reset_index(drop=True)
post_2010.to_excel(base_path+"Affiliations/aff_post_2010.xlsx", index=False)
post_2010.to_pickle(base_path+"Affiliations/aff_post_2010.pkl")

In [1199]:
combined=pd.concat([post_2010, pre_2010_df], axis=0).reset_index(drop=True)

In [1200]:


    # {"id":,"auth_num":,"aff_num":,"affiliation":"", "aff_cleaner":"","aff_cleaner_final":"","aff_main_final":""}


In [1201]:
combined['manual']=combined['manual'].fillna("NA")
combined['screened']=combined['screened'].fillna('NA')
combined.columns
combined["id"]=combined["id"].astype(str)

In [1202]:
combined.shape

(57265, 36)

In [1203]:
print(len(combined['id'].unique()))
print(len(pre_2010_df['id'].unique()))
print(len(post_2010['id'].unique()))

29838
25619
4220


In [1204]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [1205]:
replacements=pd.read_excel("refactor_affs_checked.xlsx")

concat_candidate=[]
for i in replacements.index:
    j=combined.index[(combined['id']==replacements.loc[i,"id"])&(combined['auth_num']==replacements.loc[i,"auth_num"])&(combined['aff_num']==replacements.loc[i,"aff_num"])]
    if len(j)==0:
        temp=combined[(combined['id']==str(replacements.loc[i,"id"]))][[
            'id', 'year', 'title', 'journal', 'pages', 'vol', 'number', 'auth_count', 'auth_list',
            'authors', 'content_type', 'stable_url']].drop_duplicates().reset_index(drop=True)
        comb=temp.iloc[0].to_dict()|replacements.iloc[i].to_dict()
        concat_candidate.append(comb)
        
        # print(pd.concat([temp,replacements.iloc[i]], axis=1).shape)
        # print(replacements.iloc[i].shape)
    else:
        combined.loc[j[0], 'aff_main_final']=replacements.loc[i,'aff_main_final']


combined=pd.concat([combined,pd.DataFrame(concat_candidate)],axis=0).reset_index(drop=True)

In [1206]:
combined.shape

(57274, 36)

In [1207]:
repl={
    'é':'e',
    "ä":'a',
    "à":'a',
    'è':'e',
    'á':'a',
    'ö':'o',
    'ü':'u',
    'ó':'o',
    'í':'i',
    'ğ':'g',
    "&":'and',
    "u.s.":'us',
    "suny at": "state university of new york -",
    'd.c.':'dc',
    'ç':'c',
    '\xa0':' ',
    'ń':'n',
    '–':'-',
    '’':"'",
    'ò':'o', 
    'ò':'o',
    'ã':'a',
    'ñ':'n',
    'ú':'u', 
    'ø':'o', 
    'ê':'e'
}
combined["aff_main_final_repl"]=combined['aff_main_final']

for i in repl.keys():
    combined["aff_main_final_repl"]=combined["aff_main_final_repl"].str.lower().str.replace(i,repl[i])


In [1208]:
s=list(combined[combined["aff_main_final_repl"].isna()==False]["aff_main_final_repl"].unique())
s.sort()
len(s)

3041

In [1209]:
allowed="abcdefghijklmnopqrstuvwxyz- ,'.()1234567890"

In [1210]:
exclude=[]
for i in s:
    whole=False
    for j in i:
        if j not in allowed:
            whole=True
            exclude.append(j)
            print('"'+j+'",')
    if whole==True:
        print(i)

"/",
brazilian school of public and business administration getulio vargas foundation - ebape/fgv
"/",
centre for population, poverty and public policy studies - ceps/instead
"/",
ceps/instead
";",
hebrew university of jerusalem;federmann centre for the study of rationality
"/",
ipea/inpes
"!",
yahoo! research


In [1211]:
consol_fin={}
with open('consol_fin.json', 'r', encoding="utf-8") as f:
    consol_fin = json.load(f)


consol_fin_vals=[]
for i in consol_fin.values():
    consol_fin_vals+=i

for i in s:
    if i in consol_fin.keys():
        if i in consol_fin[i]:
            continue
        else:
            print("{a: \""+i+"\"}")
    else:
        if i in consol_fin_vals:
            continue
        else: 
            print("{b: \""+i+"\"}")

consol_fin_un={}
for i in consol_fin.keys():
    for j in consol_fin[i]:
        if j in consol_fin_un.keys():
            if (consol_fin_un[j]!=i):
                print("duplicate: "+j)
        consol_fin_un[j]=i

In [1212]:
combined["aff_main_final_repl_consol"]=combined["aff_main_final_repl"]

In [1213]:
for i in combined.index:
    if pd.isna(combined.loc[i, "aff_main_final_repl_consol"]):
        combined.loc[i, "aff_main_final_repl_consol"]="none"
    else:
        combined.loc[i, "aff_main_final_repl_consol"]=consol_fin_un[combined.loc[i, "aff_main_final_repl"]]

In [1214]:
len(combined["aff_main_final_repl_consol"].unique())

2482

In [1215]:
aff_country=pd.read_csv("aff_country_match.csv", quotechar='"')

In [1216]:
aff_country["affiliation"]=aff_country["affiliation"].str.strip()
aff_country["country"]=aff_country["country"].str.strip()

In [1217]:
aff_country_un={}
for i in aff_country.index:
    aff_country_un[aff_country.loc[i,"affiliation"]]=aff_country.loc[i,"country"]

In [1218]:
combined["country"]=""
for i in combined.index:
    combined.loc[i, "country"]=aff_country_un[combined.loc[i, "aff_main_final_repl_consol"]]

In [1219]:
consol_fin

{'a b laffer associates': ['a b laffer associates'],
 'aalto university': ['aalto university school of business',
  'aalto university',
  'department of economics, american university'],
 'aarhus denmark (city)': ['aarhus denmark (city)'],
 'aarhus university': ['school of economics and management, aarhus university',
  'department of economics and business, department of economics and business economics, aarhus university, denmark',
  'aarhus university',
  'center for research in econometric analysis of time series (creates)',
  'interdisciplinary center for organizational architecture - icoa, aarhus university',
  'university of aarhus',
  'icoa',
  'dale t. mortensen centre',
  'university of arhus',
  'àarhus universitet',
  'creates, aarhus university',
  'department of economics and business economics, aarhus university, denmark',
  'department of economics and business economics, aarhus university'],
 'abdul latif jameel poverty action lab': ['abdul latif jameel poverty action 

In [1220]:
for i in consol_fin.keys():
    consol_fin[i]=list(set(consol_fin[i]))

In [1221]:
len(set(consol_fin.keys()))

2505

In [1222]:
journals={
    'quarterly journal of economics': "QJE" , 
    'econometrica':'ECTA',
    'review of economic studies':'RES', 
    'journal of political economy':"JPE",
    'american economic review':'AER', 
    'RES':'RES', 
    'QJE':"QJE", 
    'JPE':"JPE", 
    'AER':"AER",
    'ECONOMETRICA':'ECTA', 
    'Journal of Political Economy':"JPE",
    'The American Economic Review':'AER',
    'The Quarterly Journal of Economics':"QJE", 
    'Econometrica':'ECTA',
    'The Review of Economic Studies':'RES'
}

for i in journals.keys():
    combined.loc[combined["journal"]==i, "journal"]=journals[i]

In [1223]:
combined.columns

Index(['id', 'year', 'title', 'journal', 'pages', 'vol', 'number', 'author',
       'affiliation', 'auth_num', 'aff_num', 'aff_cleaner',
       'aff_cleaner_final', 'aff_main_final', 'aff_subunit_final',
       'aff_department_final', 'alt_final', 'country', 'is_business_school',
       'is_institute', 'manual', 'screened', 'auth_count', 'auth_list',
       'authors', 'content_type', 'stable_url', 'ref_type', 'ref_string',
       'ref_string_cleaner', 'pdf_url', 'hitId', 'worker', 'aff_clean',
       'aff_main', 'add_sub', 'aff_main_final_repl',
       'aff_main_final_repl_consol'],
      dtype='object')

In [1224]:
for i in consol_fin.keys():
    for j in consol_fin.keys():
        if (i in consol_fin[j]) & (i!=j):
            print('"'+i+'"')

In [1225]:
for i in combined.index:
    if combined.loc[i,"aff_main_final_repl"] in consol_fin.keys():
        continue
    elif combined.loc[i,"aff_main_final_repl"] in consol_fin_vals:
        continue
    else:
        print('"'+str(combined.loc[i, "aff_main_final_repl"])+'"')

"nan"
"nan"
"nan"
"nan"
"nan"
"nan"
"nan"
"nan"
"nan"
"nan"
"nan"
"nan"
"nan"
"nan"
"nan"
"nan"
"nan"
"nan"
"nan"
"nan"
"nan"


In [1226]:
for i in combined.index:
    if combined.loc[i,'author'].strip()[0]=="'":
        combined.loc[i,'author']=combined.loc[i,'author'].strip()[1:].strip()
    if combined.loc[i,'author'].strip()[-1]=="'":
        combined.loc[i,'author']=combined.loc[i,'author'].strip()[0:-1].strip()

combined['year']=combined['year'].astype(int)

combined["aff_main_final_og"]=combined["aff_main_final"]
combined["aff_main_final"]=combined["aff_main_final_repl_consol"]

In [ ]:
combined.to_excel(base_path+"Affiliations/affiliations_combined.xlsx", index=False)
combined.to_pickle(base_path+"Affiliations/affiliations_combined.pkl")

sub=combined[['id', 'year','author',
       'auth_num', 'aff_num', 'aff_main_final', 'country']]
sub.to_excel(base_path+"Affiliations/affiliations_combined_sub.xlsx", index=False)
sub.to_pickle(base_path+"Affiliations/affiliations_combined_sub.pkl")

In [1228]:
combined.columns

Index(['id', 'year', 'title', 'journal', 'pages', 'vol', 'number', 'author',
       'affiliation', 'auth_num', 'aff_num', 'aff_cleaner',
       'aff_cleaner_final', 'aff_main_final', 'aff_subunit_final',
       'aff_department_final', 'alt_final', 'country', 'is_business_school',
       'is_institute', 'manual', 'screened', 'auth_count', 'auth_list',
       'authors', 'content_type', 'stable_url', 'ref_type', 'ref_string',
       'ref_string_cleaner', 'pdf_url', 'hitId', 'worker', 'aff_clean',
       'aff_main', 'add_sub', 'aff_main_final_repl',
       'aff_main_final_repl_consol', 'aff_main_final_og'],
      dtype='object')